In [ ]:
import pandas as pd
import datetime
import requests
import json
import math
import numpy as np

## key 저장

In [ ]:
f = open("keyfile.txt", 'r')
key = {}

for i in range(1, 11):
    key[i] = f.readline().rstrip()
    print('key'+str(i))
f.close()

key

## 데이터 목록
### 1. 영화 목록
* INPUT
    - 키 값, 현재 페이지, 결과 ROW 의 개수, 영화명, 감독명, 조회시작 개봉연도(YYYY), 조회종료 개봉연도(YYYY), 조회시작 제작연도(YYYY), 조회종료 제작연도(YYYY), 국적코드, 영화유형코드

* OUTPUT
    - 영화코드, 영화명(국문),영화명(영문), 제작연도, 개봉연도, 영화유형, 제작상태, 제작국가(전체), 영화장르(전체), 대표 제작국가명, 대표 장르명, 영화감독, 영화감독명, 제작사, 제작사 코드, 제작사명

### 2. 영화 상세정보
* INPUT
    - 키 값, 영화코드
    
* OUTPUT
    - 영화코드, 영화명(국문), 영화명(영문), 영화명(원문), 제작연도, 상영시간, 개봉연도, 제작상태명, 영화유형명, 제작국가, 제작국가명, 장르명, 감독, 감독명, 감독명(영문), 배우, 배우명, 배우명(영문), 배역명, 배역명(영문), 상영형태 구분1, 상영형태 구분2, 상영형태명, 심의정보, 심의번호, 관람등급 명칭, 참여 영화사, 참여 영화사 코드, 참여 영화사명, 참여 영화사명(영문), 참여 영화사 분야명, 스텝, 스텝명, 스텝명(영문), 스텝역할명
    
### 3. 영화사 목록
* INPUT
    - 키 값, 현재 페이지(default : “1”), 결과 ROW 의 개수(default : “10”), 영화사명, 대표자명, 분류코드(default: 전체)
* OUTPUT
    - 영화사 코드, 영화사명, 영화사명(영문), 영화사 분류, 대표자명, 필모리스트

### 4. 영화사 상세정보
* INPUT
    - 키 값, 영화사코드
* OUTPUT
    - 영화사 코드, 영화사명, 영화사명(영문), 대표자명, 영화사 분류, 영화사 분류명, 영화사 필모, 참여 영화코드, 참여 영화명, 영화사 참여 분류명

### 5. 영화인 목록
* INPUT
    - 키 값, 현재 페이지를 지정(default : “1”), 결과 ROW 의 개수(default : “10”), 영화인명, 필모리스트
* OUTPUT
    - 영화인 코드, 영화인명, 영화인명(영문), 분야, 필모리스트

### 6. 영화인 상세정보
* INPUT
    - 키 값, 영화인코드
* OUTPUT
    - 영화인 코드, 영화인명, 영화인명(영문), 성별, 영화인 분류명, 영화인 필모, 참여 영화코드, 참여 영화명, 참여 분야, 관련 URL


## 1. 영화 리스트들 중에서 2019 박스오피스로 필터링 된 영화들 정보 가져오기(xlsx)

상영 횟수 40회 이하 아웃라이어 제거 위함(영진위 한국영화성평등소위원회 분석 참고). 


In [3]:
movies_2015_filtered = pd.read_excel('Input_Data/KOBIS_boxoffice_2015_filtered.xlsx')
# movies_2015_filtered = movies_2015_filtered[:-1] 합계 수동으로 지움
movies_2015_filtered

,순위,영화명,개봉일,매출액,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우
0,1,베테랑,2015-08-05,105168155250,0.120,105168155250,13414009,13414009,1115,199231,한국,한국,"(주)외유내강,(주)필름케이",씨제이이앤엠(주),15세이상관람가,"액션,드라마",류승완,"황정민,유아인,유해진,오달수,장윤주,김시후,오대환,정웅인,정만식,송영창,진경,유인영..."
1,2,암살,2015-07-22,98463132781,0.112,98463132781,12705700,12705700,1519,175185,한국,한국,(주)케이퍼필름,(주)쇼박스,15세이상관람가,"액션,드라마",최동훈,"전지현,이정재,하정우,오달수,조진웅,이경영,최덕문,김의성,박병은,김홍파,강용빈,홍성..."
2,4,내부자들,2015-11-19,56465665657,0.064,56465665657,7055332,7055332,1129,152265,한국,한국,(유)내부자들 문화전문회사,(주)쇼박스,청소년관람불가,"범죄,드라마",우민호,"이병헌,조승우,백윤식,이경영,김홍파,배성우,조재윤,김대명,조우진,이엘,정만식,김병옥..."
3,7,검은 사제들,2015-11-05,42405282092,0.048,42405282092,5442128,5442128,1109,131155,한국,한국,영화사 집,씨제이이앤엠(주),15세이상관람가,"미스터리,드라마",장재현,"김윤석,강동원,박소담,김병옥,김의성,이호재,남일우,김동현,김미영,김혜민,박지수,송선..."
4,5,사도,2015-09-16,48842902501,0.056,48842902501,6246849,6246849,1210,124246,한국,한국,(주)타이거픽쳐스,(주)쇼박스,12세이상관람가,사극,이준익,"송강호,유아인,문근영,전혜진,김해숙,박원상,진지희,박소담,서예지,이효제,곽동호,최희..."
5,6,연평해전,2015-06-24,45563228330,0.052,45563228330,6043784,6043784,1013,114450,한국,한국,(주)로제타시네마,(주)넥스트엔터테인먼트월드(NEW),12세이상관람가,드라마,김학순,"김무열,진구,김율호,이현우,이완,이청아,천민희,이한종,박찬희,박혜준,최순조,최승준,..."
6,10,스물,2015-03-25,23563421686,0.027,23563421686,3044811,3044811,926,100161,한국,한국,"(주)영화나무,(주)아이에이치큐",(주)넥스트엔터테인먼트월드(NEW),15세이상관람가,"코미디,드라마",이병헌,"김우빈,이준호,강하늘,정덕희,주창욱,권지훈,오하늬,오현경,이성두,정소민,안재홍,이유..."
7,8,히말라야,2015-12-16,39469775448,0.045,39469775448,5129409,5129409,1095,81042,한국,한국,(주)제이케이필름,씨제이이앤엠(주),12세이상관람가,드라마,이석훈,"황정민,정우,조성하,김인권,라미란,김원해,이해영,전배수,정유미,유선,한명환,김재준,..."
8,9,조선명탐정 : 사라진 놉의 딸,2015-02-11,30456879428,0.035,30456879428,3872015,3872015,825,80499,한국,한국,청년필름(주),(주)쇼박스,12세이상관람가,"사극,코미디,어드벤처",김석윤,"김명민,오달수,이연희,강다현,전건익,정명철,박지선,배영민,정혜은,조관우,조명행,최지..."
9,11,극비수사,2015-06-18,22389535897,0.025,22389535897,2860786,2860786,894,66211,한국,한국,"(주)제이콘컴퍼니,(주)영화사 신세계",(주)쇼박스,15세이상관람가,"드라마,범죄",곽경택,"김윤석,유해진,송영창,장영남,정호빈,이준혁,이종일,장성훈,정규용,정기용,백승호,이효..."


In [41]:
movies_2015_filtered.isnull().sum()

순위        0
영화명       0
개봉일       0
매출액       0
매출액       0
누적매출액     0
관객수       0
누적관객수     0
스크린수      0
상영횟수      0
대표국적      0
국적        0
제작사       0
배급사       0
등급        0
장르        0
감독        0
배우        8
dtype: int64

In [78]:
# Null 값 없음
# movies_2015_filtered.fillna('-', inplace=True)

In [80]:
# Null 값 없음
# movies_2015_filtered[movies_2015_filtered['감독 '] == '-']

,순위,영화명,개봉일,매출액,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우
96,35,러브 유어셀프 인 서울,2019-01-26,3209682900,0.004,3209682900,342366,342366,224,4535,한국,한국,"빅히트 엔터테인먼트,씨제이 씨지브이(CJ CGV)(주)",씨제이 씨지브이(CJ CGV)(주),전체관람가,공연,-,"김남준,김석진,민윤기,정호석,박지민,김태형,전정국"


### 영화 기본정보 가져오는 함수

In [45]:
# 페이지 수, 조회시작 개봉연도, 조회종료 개봉연도를 파라미터(입력값)로 지정해준다.
def get_movies(movieNm, year, directorNm):
    # 영화 목록 API JSON URL
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json'
    
    # 입력받은 영화이름, 년도, 감독이름으로 API 호출.
    payload = {
    'key': key5
    , 'itemPerPage': '100' # 한페이지에 한번에 불러올 수 있는 최대 데이터의 갯수
    , 'movieNm':movieNm
    , 'directorNm':directorNm
    , 'openStartDt': year
    }

    # 결과를 변수에 담고 json 형식으로 변환해준다.
    res = requests.get(url, params=payload).json()
    
    # 쓸데 없는 겉부분 버리고 영화 리스트 컨텐츠만 가져옴
    return res['movieListResult']['movieList']

In [46]:
# 함수 테스트
res = get_movies('스물', 2015, '이병헌')

In [47]:
pd.DataFrame(res)

,companys,directors,genreAlt,movieCd,movieNm,movieNmEn,nationAlt,openDt,prdtStatNm,prdtYear,repGenreNm,repNationNm,typeNm
0,"[{'companyCd': '20154488', 'companyNm': '(주)영화...",[{'peopleNm': '이병헌'}],"코미디,드라마",20148845,스물,Twenty,한국,20150325,개봉,2014,코미디,한국,장편


### 영화 기본정보 가져오는 함수 돌려서 영화 코드 가져오기

In [48]:
# 영화 몇개나 있는지 확인
len(movies_2015_filtered)

164

In [49]:
# 컬럼 이름 틀리지 않도록 확인. *'감독 '* 주의
movies_2015_filtered.columns

Index(['순위', '영화명', '개봉일', '매출액 ', '매출액', '누적매출액 ', '관객수 ', '누적관객수 ', '스크린수 ',
       '상영횟수 ', '대표국적 ', '국적 ', '제작사 ', '배급사 ', '등급 ', '장르 ', '감독 ', '배우 '],
      dtype='object')

In [50]:
# 함수에 필요한 정보(영화명, 감독명, 개봉일) 가져오기
movieNm_list = movies_2015_filtered['영화명']
movieNm_list

0                         베테랑
1                          암살
2                        내부자들
3                      검은 사제들
4                          사도
5                        연평해전
6                          스물
7                        히말라야
8            조선명탐정 : 사라진 놉의 딸
9                        극비수사
10                     악의 연대기
11                 탐정 : 더 비기닝
12                        더 폰
13                    강남 1970
14                       장수상회
15                    뷰티 인사이드
16                        쎄시봉
17                     오늘의 연애
18                      차이나타운
19                         간신
20                         대호
21                       그놈이다
22                     성난 변호사
23                       살인의뢰
24                  특종: 량첸살인기
25                 열정같은소리하고있네
26                     미쓰 와이프
27                        허삼관
28                        헬머니
29                        무뢰한
                ...          
134                     성난 화가
135                     젊은 처제
136       

In [51]:
# 감독명은 한명만 있어도 검색 가능
directorNm_list = movies_2015_filtered['감독 '].apply(lambda x: x.split(',')[0])
directorNm_list

0         류승완
1         최동훈
2         우민호
3         장재현
4         이준익
5         김학순
6         이병헌
7         이석훈
8         김석윤
9         곽경택
10        백운학
11        김정훈
12        김봉주
13         유하
14        강제규
15          백
16        김현석
17        박진표
18        한준희
19        민규동
20        박훈정
21        윤준형
22        허종호
23        손용호
24         노덕
25        정기훈
26        강효진
27        하정우
28        신한솔
29        오승욱
        ...  
134       전규환
135       이세종
136       박준범
137       전병철
138       강명진
139       김철민
140        강훈
141       정단우
142       최우성
143        김선
144       장동홍
145        이난
146      최율권희
147       황상원
148       이상호
149       이영민
150       강의석
151       전성호
152       김대성
153       봉만대
154       이숭환
155    조조 히데오
156       경석호
157       탁승오
158       장성수
159       공자관
160       이상우
161       최영민
162       김동후
163       문정윤
Name: 감독 , Length: 164, dtype: object

In [52]:
# 개봉일은 년도로만 검색 가능
openYear_list = movies_2015_filtered['개봉일'].apply(lambda x: x.year)
openYear_list

0      2015
1      2015
2      2015
3      2015
4      2015
5      2015
6      2015
7      2015
8      2015
9      2015
10     2015
11     2015
12     2015
13     2015
14     2015
15     2015
16     2015
17     2015
18     2015
19     2015
20     2015
21     2015
22     2015
23     2015
24     2015
25     2015
26     2015
27     2015
28     2015
29     2015
       ... 
134    2015
135    2015
136    2015
137    2015
138    2015
139    2015
140    2015
141    2015
142    2015
143    2015
144    2015
145    2015
146    2015
147    2015
148    2015
149    2015
150    2015
151    2015
152    2015
153    2015
154    2015
155    2015
156    2015
157    2015
158    2015
159    2015
160    2015
161    2015
162    2015
163    2015
Name: 개봉일, Length: 164, dtype: int64

In [53]:
movie_info_list = []

for movie in zip(movieNm_list, openYear_list, directorNm_list):
    print(movie)
    movie_info_list.append(get_movies(movie[0], movie[1], movie[2])[0])

('베테랑', 2015, '류승완')
('암살', 2015, '최동훈')
('내부자들', 2015, '우민호')
('검은 사제들', 2015, '장재현')
('사도', 2015, '이준익')
('연평해전', 2015, '김학순')
('스물', 2015, '이병헌')
('히말라야', 2015, '이석훈')
('조선명탐정 : 사라진 놉의 딸', 2015, '김석윤')
('극비수사', 2015, '곽경택')
('악의 연대기', 2015, '백운학')
('탐정 : 더 비기닝', 2015, '김정훈')
('더 폰', 2015, '김봉주')
('강남 1970', 2015, '유하')
('장수상회', 2015, '강제규')
('뷰티 인사이드', 2015, '백')
('쎄시봉', 2015, '김현석')
('오늘의 연애', 2015, '박진표')
('차이나타운', 2015, '한준희')
('간신', 2015, '민규동')
('대호', 2015, '박훈정')
('그놈이다', 2015, '윤준형')
('성난 변호사', 2015, '허종호')
('살인의뢰', 2015, '손용호')
('특종: 량첸살인기', 2015, '노덕')
('열정같은소리하고있네', 2015, '정기훈')
('미쓰 와이프', 2015, '강효진')
('허삼관', 2015, '하정우')
('헬머니', 2015, '신한솔')
('무뢰한', 2015, '오승욱')
('순수의 시대', 2015, '안상훈')
('손님', 2015, '김광태')
('내 심장을 쏴라', 2015, '문제용')
('도리화가', 2015, '이종필')
('서부전선', 2015, '천성일')
('오피스', 2015, '홍원찬')
('경성학교: 사라진 소녀들', 2015, '이해영')
('위험한 상견례 2', 2015, '김진영')
('극적인 하룻밤', 2015, '하기호')
('소수의견', 2015, '김성제')
('협녀, 칼의 기억', 2015, '박흥식')
('치외법권', 2015, '신동엽')
('소셜포비아', 2015, '홍석재')
('

In [54]:
movie_basic_info = pd.DataFrame(movie_info_list)
movie_basic_info

,companys,directors,genreAlt,movieCd,movieNm,movieNmEn,nationAlt,openDt,prdtStatNm,prdtYear,repGenreNm,repNationNm,typeNm
0,"[{'companyCd': '20100374', 'companyNm': '(주)외유...",[{'peopleNm': '류승완'}],"액션,드라마",20148048,베테랑,Veteran,한국,20150805,개봉,2014,액션,한국,장편
1,"[{'companyCd': '20062083', 'companyNm': '(주)케이...",[{'peopleNm': '최동훈'}],"액션,드라마",20148851,암살,Assassination,한국,20150722,개봉,2015,액션,한국,장편
2,"[{'companyCd': '20157297', 'companyNm': '(유)내부...",[{'peopleNm': '우민호'}],"범죄,드라마",20149314,내부자들,Inside Men,한국,20151119,개봉,2015,범죄,한국,장편
3,"[{'companyCd': '20100450', 'companyNm': '영화사 집'}]",[{'peopleNm': '장재현'}],"미스터리,드라마",20145503,검은 사제들,The Priests,한국,20151105,개봉,2015,미스터리,한국,장편
4,"[{'companyCd': '20100239', 'companyNm': '(주)타이...",[{'peopleNm': '이준익'}],사극,20148846,사도,The Throne,한국,20150916,개봉,2014,사극,한국,장편
5,"[{'companyCd': '20062680', 'companyNm': '(주)로제...",[{'peopleNm': '김학순'}],드라마,20134798,연평해전,Northern Limit Line,한국,20150624,개봉,2015,드라마,한국,장편
6,"[{'companyCd': '20154488', 'companyNm': '(주)영화...",[{'peopleNm': '이병헌'}],"코미디,드라마",20148845,스물,Twenty,한국,20150325,개봉,2014,코미디,한국,장편
7,"[{'companyCd': '20100278', 'companyNm': '(주)제이...",[{'peopleNm': '이석훈'}],드라마,20136068,히말라야,The Himalayas,한국,20151216,개봉,2015,드라마,한국,장편
8,"[{'companyCd': '20100187', 'companyNm': '청년필름(...",[{'peopleNm': '김석윤'}],"사극,코미디,어드벤처",20149570,조선명탐정 : 사라진 놉의 딸,Detective K : Secret of the Lost Island,한국,20150211,개봉,2014,사극,한국,장편
9,"[{'companyCd': '2013214', 'companyNm': '(주)제이콘...",[{'peopleNm': '곽경택'}],"드라마,범죄",20148855,극비수사,The Classified File,한국,20150618,개봉,2014,드라마,한국,장편


<br>\
<br>
### 영화 상세정보 함수

영화코드를 가져왔으니 이제 영화 상세정보를 가져올 수 있습니다.

In [55]:
def get_movie_details(movieCd):
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json'

    payload = {
        'key': key5
        , 'movieCd': movieCd
    }

    res = requests.get(url, params=payload).json()
    
    return res['movieInfoResult']['movieInfo']

영화 기본 정보에서 알 수 있는 정보는 제외, 이외에 상세정보에서만 얻을 수 있는 데이터들 확인.

1. 러닝타임
2. 심의 정보
3. 감독 정보
4. 배우 정보
5. 제작사 정보
6. 스태프 정보

In [57]:
# get_movie_details 함수 테스트
res = get_movie_details(20145503)
res

{'movieCd': '20145503',
 'movieNm': '검은 사제들',
 'movieNmEn': 'The Priests',
 'movieNmOg': '',
 'showTm': '108',
 'prdtYear': '2015',
 'openDt': '20151105',
 'prdtStatNm': '개봉',
 'typeNm': '장편',
 'nations': [{'nationNm': '한국'}],
 'genres': [{'genreNm': '미스터리'}, {'genreNm': '드라마'}],
 'directors': [{'peopleNm': '장재현', 'peopleNmEn': 'JANG Jae-hyun'}],
 'actors': [{'peopleNm': '김윤석',
   'peopleNmEn': 'KIM Yun-seok',
   'cast': '',
   'castEn': ''},
  {'peopleNm': '강동원', 'peopleNmEn': 'GANG Dong-won', 'cast': '', 'castEn': ''},
  {'peopleNm': '박소담', 'peopleNmEn': 'PARK So-dam', 'cast': '영신', 'castEn': ''},
  {'peopleNm': '김병옥',
   'peopleNmEn': 'KIM Byung-ok',
   'cast': '박교수',
   'castEn': ''},
  {'peopleNm': '김의성',
   'peopleNmEn': 'KIM Eui-sung',
   'cast': '학장신부',
   'castEn': ''},
  {'peopleNm': '이호재', 'peopleNmEn': '', 'cast': '', 'castEn': ''},
  {'peopleNm': '남일우', 'peopleNmEn': 'NAM Il-u', 'cast': '수도원장', 'castEn': ''},
  {'peopleNm': '김동현', 'peopleNmEn': '', 'cast': '최부제 대역', 'castE

### 후가공을 위한 데이터 확인

영화 상세정보 호출 결과 그대로 데이터프레임화 할 수 없으니 필요한 정보들을 종류별로 가져와서 따로따로 저장해서 테이블화 해야겠다.

1. 러닝타임

In [58]:
res['showTm']

'108'

2. 심의 정보

In [59]:
res['audits']

[{'auditNo': '2015-MF01188 ', 'watchGradeNm': '15세이상관람가'}]

3. 감독 정보
4. 배우 정보
5. 제작사 정보
6. 스태프 정보

In [60]:
res['directors']

[{'peopleNm': '장재현', 'peopleNmEn': 'JANG Jae-hyun'}]

In [61]:
res['actors']

[{'peopleNm': '김윤석', 'peopleNmEn': 'KIM Yun-seok', 'cast': '', 'castEn': ''},
 {'peopleNm': '강동원', 'peopleNmEn': 'GANG Dong-won', 'cast': '', 'castEn': ''},
 {'peopleNm': '박소담', 'peopleNmEn': 'PARK So-dam', 'cast': '영신', 'castEn': ''},
 {'peopleNm': '김병옥',
  'peopleNmEn': 'KIM Byung-ok',
  'cast': '박교수',
  'castEn': ''},
 {'peopleNm': '김의성',
  'peopleNmEn': 'KIM Eui-sung',
  'cast': '학장신부',
  'castEn': ''},
 {'peopleNm': '이호재', 'peopleNmEn': '', 'cast': '', 'castEn': ''},
 {'peopleNm': '남일우', 'peopleNmEn': 'NAM Il-u', 'cast': '수도원장', 'castEn': ''},
 {'peopleNm': '김동현', 'peopleNmEn': '', 'cast': '최부제 대역', 'castEn': ''},
 {'peopleNm': '김미영', 'peopleNmEn': '', 'cast': '리포터2', 'castEn': ''},
 {'peopleNm': '김혜민', 'peopleNmEn': '', 'cast': '리포터1', 'castEn': ''},
 {'peopleNm': '박지수',
  'peopleNmEn': 'PARK Ji-su',
  'cast': '중국어 목소리',
  'castEn': ''},
 {'peopleNm': '송선찬',
  'peopleNmEn': 'Song Seon-chan',
  'cast': '문지기수도승',
  'castEn': ''},
 {'peopleNm': '이상민',
  'peopleNmEn': 'LEE SANGMIN',


In [62]:
res['companys']

[{'companyCd': '20100450',
  'companyNm': '영화사 집',
  'companyNmEn': 'ZIP CINEMA',
  'companyPartNm': '제작사'},
 {'companyCd': '20110854',
  'companyNm': '씨제이이앤엠(주)',
  'companyNmEn': 'CJ ENM Corp.',
  'companyPartNm': '배급사'},
 {'companyCd': '20100410',
  'companyNm': '오퍼스픽쳐스(유)',
  'companyNmEn': 'OPUS Pictures',
  'companyPartNm': '제공'},
 {'companyCd': '20110854',
  'companyNm': '씨제이이앤엠(주)',
  'companyNmEn': 'CJ ENM Corp.',
  'companyPartNm': '제공'},
 {'companyCd': '20137115',
  'companyNm': '(주)인터파크',
  'companyNmEn': '',
  'companyPartNm': '공동제공'},
 {'companyCd': '20100906',
  'companyNm': '유나이티드픽처스(주)',
  'companyNmEn': 'United Pictures',
  'companyPartNm': '해외세일즈사'}]

In [63]:
res['staffs']

[{'peopleNm': '김주형', 'peopleNmEn': 'KIM Joo-hyung', 'staffRoleNm': '투자'},
 {'peopleNm': '이유진', 'peopleNmEn': 'LEE Eugene', 'staffRoleNm': '제작'},
 {'peopleNm': '백지선', 'peopleNmEn': 'BAEK Ji-seon', 'staffRoleNm': '프로듀서'},
 {'peopleNm': '오효진', 'peopleNmEn': '', 'staffRoleNm': '프로듀서'},
 {'peopleNm': '송대찬', 'peopleNmEn': 'SONG Dae-chan', 'staffRoleNm': '프로듀서'},
 {'peopleNm': '이하영', 'peopleNmEn': '', 'staffRoleNm': '라인프로듀서'},
 {'peopleNm': '이정우', 'peopleNmEn': '', 'staffRoleNm': '제작팀'},
 {'peopleNm': '이상수', 'peopleNmEn': '', 'staffRoleNm': '제작팀'},
 {'peopleNm': '이가람', 'peopleNmEn': '', 'staffRoleNm': '제작팀'},
 {'peopleNm': '윤석동', 'peopleNmEn': '', 'staffRoleNm': '제작팀'},
 {'peopleNm': '전예진', 'peopleNmEn': '', 'staffRoleNm': '제작팀'},
 {'peopleNm': '장주연', 'peopleNmEn': '', 'staffRoleNm': '제작팀'},
 {'peopleNm': '장재하', 'peopleNmEn': '', 'staffRoleNm': '제작팀'},
 {'peopleNm': '정준혁', 'peopleNmEn': '', 'staffRoleNm': '제작팀'},
 {'peopleNm': '김도균', 'peopleNmEn': '', 'staffRoleNm': '제작팀'},
 {'peopleNm': '황상길

### 후가공

1. 상세정보를 확인하고 싶은 영화리스트에서 영화 코드만 뽑아옵니다.
2. for문 돌면서 get_movie_details로 API 호출.
3. 각각의 영화에 대한 API 호출 결과를 받아서 필요한 정보들을 데이터프레임화 할 수 있는 형태로 리스트에 각각 저장
4. 필요한 경우에 영화 코드로 조인해서 하나의 데이터로 사용할 수 있게 각각의 데이터와 영화코드 함께 딕셔너리 형태로 저장

In [65]:
# 영화 코드만 뽑아오기
movieCd_list = movie_basic_info['movieCd']

len(movieCd_list)

164

In [66]:
movie_showTm = []
movie_audits = []
movie_directors = []
movie_actors = []
movie_staffs = []
movie_companys = []

# 테스트 할 때는 `movieCd_list.iloc[:1]` 하나만 돌려보세요.

# movieCd_list 길이가 3000이 넘어갈 경우 for문을 쪼개서 돌려야 합니다. 
# 함수 정의된 부분으로 가서 key도 바꿔줘야 해요!.
# ex) movieCd_list.iloc[:3000] 이런 식으로 구간을 나눠서 여러번 함수를 실행시켜야겠죠?
for code in movieCd_list:
    res = get_movie_details(code)
    movieNm = res['movieNm']
    print(code)
    

    showTm_dict = {}
    showTm_dict['movieCd'] = code
    showTm_dict['showTm'] = res['showTm']
    movie_showTm.append(showTm_dict)

    for audit in res['audits']:
        audits_dict = {}
        audits_dict['movieCd'] = code
        audits_dict['auditNo'] = audit['auditNo']
        audits_dict['watchGradeNm'] = audit['watchGradeNm']
        movie_audits.append(audits_dict)
    
    for director in res['directors']:
        person_info = {}
        person_info['movieCd'] = code
        person_info['movieNm'] = movieNm
        person_info['peopleNm'] = director['peopleNm']
        person_info['repRoleNm'] = '감독'
        movie_directors.append(person_info)
        
    for actor in res['actors']:
        person_info = {}
        person_info['movieCd'] = code
        person_info['movieNm'] = movieNm
        person_info['peopleNm'] = actor['peopleNm']
        person_info['repRoleNm'] = '배우'
        person_info['cast'] = actor['cast']
        movie_actors.append(person_info)
        
    for staff in res['staffs']:
        person_info = {}
        person_info['movieCd'] = code
        person_info['movieNm'] = movieNm
        person_info['peopleNm'] = staff['peopleNm']
        person_info['staffRoleNm'] = staff['staffRoleNm']
        movie_staffs.append(person_info)
    
    for company in res['companys']:
        person_info = {}
        person_info['movieCd'] = code
        person_info['companyCd'] = company['companyCd']
        person_info['companyNm'] = company['companyNm']
        person_info['companyPartNm'] = company['companyPartNm']
        movie_companys.append(person_info)

20148048
20148851
20149314
20145503
20148846
20134798
20148845
20136068
20149570
20148855
20149569
20156557
20157464
20136888
20140703
20143572
20148188
20142407
20149266
20142402
20145171
20157465
20142855
20148461
20145504
20156748
20143513
20131262
20148841
20130566
20136063
20148844
20148553
20140144
20140193
20143511
20141224
20143512
20157501
20136057
20130021
20155826
20141602
20136869
20133682
20149062
20143510
20156807
20142856
20147947
20145501
20100211
20149961
20158481
20141060
20156553
20151782
20144442
20156441
20156321
20149264
20157587
20156281
20147803
20141598
20157986
20143502
20151582
20140241
20150025
20159161
20147130
20153422
20135264
20141702
20153801
20153311
20158701
20157431
20141612
20154323
20152870
20156047
20141002
20140190
20134800
20158951
20148449
20148982
20141705
20141608
20158682
20127568
20148676
20145202
20148685
20133803
20143717
20144601
20141610
20149298
20153307
20124398
20148631
20142403
20152064
20163218
20133531
20144422
20141110
20130260
2

추려진 데이터들 확인

In [73]:
showTm_df = pd.DataFrame(movie_showTm)
showTm_df.to_csv('Output_Data/showTm_2015_ko.csv', index=False)

In [74]:
audits_df = pd.DataFrame(movie_audits)
audits_df.to_csv('Output_Data/audits_2015_ko.csv', index=False)

In [75]:
directors_df = pd.DataFrame(movie_directors)
directors_df.to_csv('Output_Data/directors_2015_ko.csv', index=False)

In [76]:
actors_df = pd.DataFrame(movie_actors)
actors_df.to_csv('Output_Data/actors_2015_ko.csv', index=False)

In [77]:
staffs_df = pd.DataFrame(movie_staffs)
staffs_df.to_csv('Output_Data/staffs_2015_ko.csv', index=False)

In [78]:
companys_df = pd.DataFrame(movie_companys)
companys_df.to_csv('Output_Data/companys_2015_ko.csv', index=False)

### # 주의 # 한 테스크가 끝날 때마다 csv로 저장하고 다음 작업 시작할 때는 저장된 csv 불러와서 시작해야 합니다.

In [8]:
directors_df = pd.read_csv('Output_Data/directors_2015_ko.csv')

print(directors_df.shape)
directors_df.sample()

(178, 4)


,movieCd,movieNm,peopleNm,repRoleNm
145,20140191,플랑크 상수,조성규,감독


In [7]:
actors_df = pd.read_csv('Output_Data/actors_2015_ko.csv')

print(actors_df.shape)
actors_df.sample()

(2388, 5)


,cast,movieCd,movieNm,peopleNm,repRoleNm
1278,토이앤죠이직원,20147947,워킹걸,배유람,배우


In [9]:
staffs_df = pd.read_csv('Output_Data/staffs_2015_ko.csv')

writers_df = staffs_df[staffs_df['staffRoleNm'] == '시나리오(각본)']

print(writers_df.shape)
writers_df.sample()

(194, 4)


,movieCd,movieNm,peopleNm,staffRoleNm
3970,20156557,탐정 : 더 비기닝,김정훈,시나리오(각본)


In [5]:
companys_df = pd.read_csv('Output_Data/companys_2015_ko.csv')

<br>\
<br>\
<br>
## 영화인 정보 가져오기

위에서 영화 상세정보 데이터에서 감독, 배우, 스태프 각각의 정보를 가져왔죠. 그 정보를 이용해서 영화인의 코드를 알아낼 겁니다. 번거롭지만 영화인 코드는 영화인 세부정보를 가져오기 위해 꼭 필요한 데이터에요.

In [14]:
def get_staffs(movieNm, peopleNm):
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json'

    # 영화인의 기본정보를 가져오기 위해서는 정보를 알고 싶은 사람의 '이름'과 '참여했던 작품 이름'이 있어야 합니다.
    # 영화 이름만으로 검색하면 영화에 참여했던 모든 스태프를 가져올 수 있지만 동명의 영화가 함께 검색되어 특정 영화의 정보를 가져오기에는 무리가 있습니다.
    # 때문에 특정하게 알고싶은 사람이 이름과 작품의 이름으로 특정 사람을 한정지어 가져오는 것이 낫다고 판단하여 짜여진 함수입니다.
    payload = {
        'key': key[5]
        , 'curPage': 1 # 한사람만 검색되므로 1페이지를 넘어가는 경우가 없을 거라고 가정하고 진행
        , 'itemPerPage': '50' # 한사람만 검색되므로 데이터 갯수가 50개를 넘어가는 경우가 없을 거라고 가정하고 진행
        , 'peopleNm': peopleNm
        , 'filmoNames': movieNm
    }

    res = requests.get(url, params=payload).json()
    
    return res['peopleListResult']

In [15]:
def get_staffs_code(data, listNm):

    num = 0
    
    # 이름과 작품으로 영화인 정보에 접근할 것을 알고 있었기 때문에 
    # 위에서 감독, 배우, 스태프들의 정보를 담을 때 
    # 각각의 사람들이 참여했던 작품의 이름도 함께 저장이 되어있습니다.
    # 때문에 directors_df, actors_df, staffs_df 어느것을 넣더라도 똑같이 작동할 수 있어요.
    
    # get_staffs_code()에 directors_df가 입력되면
    # 아래 반복문에서 이름 컬럼과 작품 컬럼을 각각 하나씩 튜플로 묶어 반복문을 실행합니다.

    for movieNm, peopleNm in zip(data['movieNm'], data['peopleNm']):
        person_info = {}
        
    # 한번 for문이 돌때마다 코드를 검색할 사람의 이름과 작품이 뽑히고
    # , 해당 정보를 get_staffs() 함수에 집어넣으면 해당 영화인의 기본정보를 얻을 수 있습니다.
        staff = get_staffs(movieNm, peopleNm)['peopleList'][0]

        person_info['peopleCd'] = staff['peopleCd']
        person_info['movieNm'] = movieNm
        person_info['peopleNm'] = peopleNm
        person_info['repRoleNm'] = staff['repRoleNm']
        person_info['filmoNames'] = staff['filmoNames']

        listNm.append(person_info)
        
        print(peopleNm, num)
        num += 1

    return ''

In [16]:
directors_df[:3]

,movieCd,movieNm,peopleNm,repRoleNm
0,20148048,베테랑,류승완,감독
1,20148851,암살,최동훈,감독
2,20149314,내부자들,우민호,감독


In [17]:
#test
test = []
res = get_staffs_code(directors_df[:3], test)
res

류승완 0
최동훈 1
우민호 2


''

### TODO. get_staffs_code() 함수를 이용해 2019 directors, actors, staffs의 영화인 코드를 가져오기.

* 각각의 데이터 프레임에서 데이터의 갯수를 보고 데이터 프레임 전체를 한 번에 돌릴지 나누어서 몇 번에 돌릴 수 있을지 판단해서 사용하는 key 조정을 해야 합니다.

* get_staffs_code() 함수의 결과로는 리스트가 반환됩니다.

ex) 한번에 돌릴 수 있을 때:
```python
result = get_staffs_code(directors_df)
```

ex) 한번에 돌릴 수 없을 때:
```python
result1 = get_staffs_code(directors_df.iloc[:3000]) # def get_staffs_code가 정의 되어있는 코드에서 key 값을 새 것으로 바꿈.
result2 = get_staffs_code(directors_df.iloc[3000:6000])

result = result1 + result2
```

Q1) 2015 감독들의 영화인 코드 가져오기

In [16]:
directorCd_list = []

directorCd_list = get_staffs_code(directors_df, directorCd_list)

류승완 0
최동훈 1
우민호 2
장재현 3
이준익 4
김학순 5
이병헌 6
이석훈 7
김석윤 8
곽경택 9
백운학 10
김정훈 11
김봉주 12
유하 13
강제규 14
백 15
김현석 16
박진표 17
한준희 18
민규동 19
박훈정 20
윤준형 21
허종호 22
손용호 23
노덕 24
정기훈 25
강효진 26
하정우 27
신한솔 28
오승욱 29
안상훈 30
김광태 31
문제용 32
이종필 33
천성일 34
홍원찬 35
이해영 36
김진영 37
하기호 38
김성제 39
박흥식 40
신동엽 41
홍석재 42
윤재구 43
임권택 44
김아론 45
권형진 46
박영균 47
임상수 48
정범식 49
권오광 50
최윤석 51
존 카프카 52
전윤수 53
김휘 54
김상진 55
박은경 56
이동하 57
홍상수 58
김대승 59
이영준 60
조치언 61
박혁지 62
조정래 63
김보경 64
안국진 65
장건재 66
박대열 67
민경조 68
박재식 69
안용훈 70
김태식 71
최경석 72
노나카 카즈미 73
우민호 74
신수원 75
이서 76
키리야 카즈아키 77
장문일 78
임흥순 79
김대창 80
임성구 81
김은주 82
김인식 83
정세교 84
김한민 85
이광국 86
허선 87
배성상 88
김진열 89
이수정 90
정일건 91
허은희 92
민병훈 93
김명준 94
김상철 95
박정범 96
이길보라 97
장희선 98
김동명 99
문제대 100
이항배 101
황윤 102
한지원 103
신연식 104
김형주 105
이옥섭 106
구교환 107
윤성호 108
강경태 109
이종훈 110
손승웅 111
이수성 112
조성규 113
김정권 114
허철 115
문희융 116
장률 117
김진홍 118
안재민 119
박진성 120
박배일 121
조은성 122
전재홍 123
이일하 124
구자환 125
김기영 126
백재호 127
안슬기 128
안선경 129
알버트 신 130
황욱 131
박민우 132
박석영 133
주호성 134
정성복 135
서태지 136
김태경 137
박진순

Q2) 2015 배우들의 영화인 코드 가져오기

In [37]:
actorCd_list = []

actorCd_list = get_staffs_code(actors_df, actorCd_list)

황정민 0
유아인 1
유해진 2
오달수 3
장윤주 4
김시후 5
오대환 6
정웅인 7
정만식 8
송영창 9
진경 10
유인영 11
박소담 12
이동휘 13
배성우 14
천호진 15
이채은 16
조덕현 17
신지수 18
권지훈 19
정윤헌 20
장한승 21
김선웅 22
김성종 23
우영택 24
김민재 25
김재현 26
김태용 27
김지수 28
박지윤 29
박지혜 30
손성민 31
장소연 32
유재명 33
배제기 34
류아벨 35
정재헌 36
안길강 37
마동석 38
김응수 39
고규필 40
서현우 41
강영묵 42
한수현 43
전지현 44
이정재 45
하정우 46
오달수 47
조진웅 48
이경영 49
최덕문 50
김의성 51
박병은 52
김홍파 53
강용빈 54
홍성지 55
지동주 56
장현석 57
장용현 58
김민지 59
김승찬 60
안태랑 61
김예은 62
김정석 63
김정호 64
김준우 65
김지원 66
박상민 67
박정환 68
조승우 69
김해숙 70
진경 71
박소은 72
김인우 73
심희섭 74
김태준 75
차지원 76
강영묵 77
정기섭 78
금새록 79
원현준 80
이병헌 81
조승우 82
백윤식 83
이경영 84
김홍파 85
배성우 86
조재윤 87
김대명 88
조우진 89
이엘 90
정만식 91
김병옥 92
김의성 93
박진우 94
남일우 95
김연지 96
김진우 97
문소리 98
박경순 99
설우신 100
손혜진 101
신정미 102
윤보배 103
이선관 104
이재남 105
이재정 106
이주아 107
신지수 108
장현준 109
최정연 110
황준혁 111
손종학 112
조덕제 113
유재명 114
조경현 115
권태원 116
강신철 117
김윤석 118
강동원 119
박소담 120
김병옥 121
김의성 122
이호재 123
남일우 124
김동현 125
김미영 126
김혜민 127
박지수 128
송선찬 129
이상민 130
이승훈 131
이현우 132
손종학 133
조수향 134
남문철 135
김수진 136
민진웅 137
송강호 138
유

류현경 1037
이채은 1038
오대환 1039
박정민 1040
이정환 1041
최철수 1042
안지혜 1043
이성진 1044
이영수 1045
최병모 1046
기주봉 1047
손수현 1048
신문성 1049
손영순 1050
박보영 1051
엄지원 1052
박소담 1053
공예지 1054
주보비 1055
박성연 1056
심희섭 1057
고원희 1058
박주희 1059
이혜리 1060
박지선 1061
조원철 1062
최영근 1063
최지명 1064
신수연 1065
김경애 1066
김대성 1067
김대호 1068
오현정 1069
김동현 1070
이다희 1071
황석영 1072
이연정 1073
이예은 1074
이은경 1075
윤정미 1076
서미지 1077
목규리 1078
금새록 1079
박성연 1080
진세연 1081
홍종현 1082
신정근 1083
임수연 1084
정명호 1085
박진수 1086
박태규 1087
조영란 1088
조용재 1089
오재호 1090
유성진 1091
이동민 1092
이상현 1093
이상협 1094
이석현 1095
차종호 1096
이철민 1097
김선영 1098
신대섭 1099
박은혜 1100
이정현 1101
김응수 1102
윤계상 1103
한예리 1104
박효주 1105
조복래 1106
박병은 1107
정수영 1108
조영균 1109
한정국 1110
김수정 1111
한지은 1112
이지훈 1113
정수연 1114
김재화 1115
박현지 1116
진선규 1117
김의성 1118
하정완 1119
윤계상 1120
유해진 1121
김옥빈 1122
이경영 1123
장광 1124
권해효 1125
손아람 1126
박주형 1127
이은미 1128
이은우 1129
김의성 1130
조복래 1131
김종수 1132
박충선 1133
박해수 1134
장리우 1135
김태준 1136
이병헌 1137
전도연 1138
김고은 1139
이경영 1140
김태우 1141
이준호 1142
배수빈 1143
문성근 1144
박슬기 1145
장윤성 1146
장은태 1147
강풍

편보승 1946
정재훈 1947
최미라 1948
김영민 1949
김정민 1950
최종원 1951
박민상 1952
박상면 1953
조안 1954
춘자 1955
김형범 1956
박해일 1957
안성기 1958
문소리 1959
한예리 1960
권혜민 1961
허철 1962
최우식 1963
정재필 1964
임철희 1965
이준동 1966
이용수 1967
윤서희 1968
오재은 1969
김동현 1970
김동현 1971
김소연 1972
김수정 1973
김여정 1974
김예은 1975
김진주 1976
안동훈 1977
신지은 1978
김희령 1979
송효정 1980
송승민 1981
류시문 1982
박상호 1983
박정환 1984
백현진 1985
백주은 1986
백종관 1987
김학선 1988
남명렬 1989
권민중 1990
김정균 1991
장두이 1992
오지호 1993
문가영 1994
김광희 1995
윤지원 1996
이상백 1997
양진영 1998
오정세 1999
조은지 2000
송삼동 2001
박재철 2002
이미도 2003
강민지 2004
한수용 2005
문성환 2006
손민식 2007
이재석 2008
최혁 2009
김범준 2010
배정화 2011
전범수 2012
이준민 2013
정넘쳐 2014
박형준 2015
안상훈 2016
유병선 2017
한을희 2018
홍성민 2019
이상미 2020
이수연 2021
이용표 2022
김종태 2023
배용근 2024
김원규 2025
김주완 2026
김형렬 2027
박영대 2028
윤명호 2029
이재근 2030
정방수 2031
조현기 2032
홍아름 2033
창조 2034
임원희 2035
양택조 2036
정의갑 2037
이종국 2038
이희우 2039
장진혁 2040
전수환 2041
정만영 2042
박은영 2043
정석 2044
정소리 2045
박지은 2046
최영일 2047
최은진 2048
윤석훈 2049
윤성옥 2050
한승완 2051
이예은 2052
이유정 2053
이은정 2054
김영옥 2055
남태부 2056
안세하 205

''

In [38]:
actorCd_list

[{'peopleCd': '10090290',
  'movieNm': '베테랑',
  'peopleNm': '황정민',
  'repRoleNm': '배우',
  'filmoNames': '인질(가제)|다만 악에서 구하소서|공작|군함도|군함도 감독판|아수라|곡성|검사외전|히말라야|베테랑|국제시장|남자가 사랑할 때|전설의 주먹|신세계|댄싱퀸|모비딕|평양성|부당거래|끝과 시작|구르믈 버서난 달처럼|그림자 살인|슈퍼맨이었던 사나이|행복|검은 집|사생결단|내 생애 가장 아름다운 일주일|너는 내 운명|천군|달콤한 인생|여자, 정혜|마지막 늑대|바람난 가족|이공|로드무비|YMCA야구단|와이키키 브라더스|쉬리|그녀|오감도'},
 {'peopleCd': '10053802',
  'movieNm': '베테랑',
  'peopleNm': '유아인',
  'repRoleNm': '배우',
  'filmoNames': '소리도 없이|국가부도의 날|버닝|좋아해줘|사도|베테랑|우아한 거짓말|우리별 일호와 얼룩소|깡철이|완득이|하늘과 바다|서양 골동 양과자점 앤티크|좋지 아니한가|우리에게 내일은 없다|씨네 21 영화토크쇼'},
 {'peopleCd': '10054128',
  'movieNm': '베테랑',
  'peopleNm': '유해진',
  'repRoleNm': '배우',
  'filmoNames': '봉오동 전투|승리호(가제)|말모이|완벽한 타인|레슬러|1987|택시운전사|공조|럭키|그놈이다|베테랑|극비수사|타짜-신의 손|해적: 바다로 간 산적|인간중독|소수의견|감기|가디언즈|간첩|미쓰GO|마마|적과의 동침|부당거래|이끼|작은 연못|죽이고 싶은|전우치|강철중: 공공의 적 1-1|트럭|권순분여사 납치사건|아들|이장과 군수|그해 여름|타짜|국경의 남쪽|왕의 남자|이대로, 죽을 순 없다(Short Time)|공공의 적 2|혈의 누|마파도|달마야, 서울 가자|바람의 전설|빙우|영어완전정복|나비|해안선|광복절 특사|라이터를 켜라|공공의 적|무사|신라의 달밤|주유소 습격사건|블랙잭|톰과 

Q3) 2015 스태프들의 영화인 코드 가져오기

In [30]:
# 시나리오만 가져오기
writerCd_list = []

get_staffs_code(writers_df, writerCd_list)

류승완 0
이기철 1
최동훈 2
우민호 3
장재현 4
조철현 5
오승현 6
이송원 7
이병헌 8
수오 9
민지은 10
김수진 11
이남규 12
한대덕 13
곽경택 14
백운학 15
황혜순 16
김정훈 17
김봉주 18
유하 19
이상현 20
김선정 21
노경희 22
박정예 23
김현석 24
박진표 25
이병헌 26
한준희 27
이윤성 28
민규동 29
박훈정 30
윤준형 31
김영갑 32
이공주 33
최관영 34
손용호 35
안영진 36
노덕 37
서유민 38
정기훈 39
김제영 40
하정우 41
김주호 42
조사무엘 43
신한솔 44
오승욱 45
김세희 46
김광태 47
최정미 48
문제용 49
유선동 50
유선동 51
김아영 52
이종필 53
김익로 54
천성일 55
최윤진 56
이해영 57
조중훈 58
하기호 59
손아람 60
박흥식 61
최아름 62
민경근 63
홍석재 64
조현민 65
윤재구 66
송윤희 67
문정원 68
이상언 69
윤필준 70
한수봉 71
박영균 72
김은숙 73
이인균 74
임상수 75
정범식 76
권오광 77
박재우 78
제임스 그레코 79
재커리 로젠블라트 80
박재우 81
애덤 비첸 82
박희강 83
김휘 84
강향숙 85
박수진 86
이동하 87
박은경 88
홍상수 89
김대승 90
조정화 91
조정화 92
신현덕 93
박상글 94
김수훈 95
최가희 96
최인희 97
조치언 98
정상선 99
안국진 100
장건재 101
이언희 102
이권 103
변혜주 104
안용훈 105
신미희 106
미야니시 타츠야 107
마스다 히사오 108
스가 요시유키 109
강민하 110
신수원 111
이서 112
장문일 113
김대창 114
최인희 115
권오현 116
신현덕 117
김인식 118
이광국 119
허선 120
박재우 121
신범숙 122
지재형 123
허은희 124
민병훈 125
김명준 126
박정범 127
김동명 128
김재환 129
손소아 130
황윤 131
신연식 132
이옥섭 133
윤성호 134
구교환 135
강경태 

''

In [19]:
writerCd_list

[{'peopleCd': '10019069',
  'movieNm': '베테랑',
  'peopleNm': '류승완',
  'repRoleNm': '감독',
  'filmoNames': '엑시트|사바하|군함도|군함도 감독판|베테랑|베를린|평양성|부당거래|해결사|다찌마와리|감독들, 김기영을 말하다|짝패|주먹이 운다(Crying Fist)|아라한 장풍대작전|오아시스|피도 눈물도 없이|죽거나 혹은 나쁘거나|타임리스|남자니까 아시잖아요?|다찌마와리|현대인|패싸움|4월 단편 상상극장-박정민 헤는 밤|신촌좀비만화|제14회 미쟝센 단편 영화제 류승완 단편 특별전2|다섯 개의 시선|유령'},
 {'peopleCd': '20128927',
  'movieNm': '암살',
  'peopleNm': '이기철',
  'repRoleNm': '시나리오(각본)',
  'filmoNames': '도청(가제)|암살|도둑들|가위|난시청지역'},
 {'peopleCd': '10072187',
  'movieNm': '암살',
  'peopleNm': '최동훈',
  'repRoleNm': '감독',
  'filmoNames': '도청(가제)|암살|도둑들|전우치|중천|타짜|소년, 천국에 가다|그때 그 사람들|범죄의 재구성|눈물|간결한 여행|스미스씨 브라질에 가다'},
 {'peopleCd': '20111283',
  'movieNm': '내부자들',
  'peopleNm': '우민호',
  'repRoleNm': '감독',
  'filmoNames': '남산의 부장들|마약왕|내부자들: 디 오리지널|내부자들|간첩|파괴된 사나이|누가 예수를 죽였는가?'},
 {'peopleCd': '10060462',
  'movieNm': '검은 사제들',
  'peopleNm': '장재현',
  'repRoleNm': '감독',
  'filmoNames': '사바하|시간위의 집|검은 사제들|광해, 왕이 된 남자|특수본|바다|12번째 보조사제|버스|인도에서 온 말리|오오단편극장 - 이학주 배우전|어둑시니: 두

In [ ]:
# 2만명이라 일단 보류
staffCd_list1 = []

# get_staffs_code(staffs_df[:3000], staffCd_list1)

In [ ]:
staffCd_list2 = []
# get_staffs_code(staffs_df[3000:6000], staffCd_list2)

In [ ]:
staffCd_list3 = []
# get_staffs_code(staffs_df[6000:], staffCd_list3)

Q4) 2019 감독, 배우, 스태프들의 영화인 코드를 하나의 데이터 프레임으로 합쳐 csv로 저장해주세요. index=False로 저장해주세요.

해당 데이터프레임을 이용해 영화인 상세 데이터를 가져와야 합니다.

각각 csv로 저장하기

In [17]:
directorCd_df = pd.DataFrame(directorCd_list)
directorCd_df.to_csv('Output_Data/directorCd_2015_list.csv', index=False)

In [39]:
actorCd_df = pd.DataFrame(actorCd_list)
actorCd_df.to_csv('Output_Data/actorCd_2015_list.csv', index=False)

In [31]:
writerCd_df = pd.DataFrame(writerCd_list)
writerCd_df.to_csv('Output_Data/writerCd_2015_list.csv', index=False)

In [ ]:
# staffCd1_df = pd.DataFrame(staffCd_list1)
# staffCd1_df.to_csv('Output_Data/staffCd_2015_list1.csv', index=False)

In [ ]:
# staffCd2_df = pd.DataFrame(staffCd_list2)
# staffCd2_df.to_csv('Output_Data/staffCd_2015_list2.csv', index=False)

In [29]:
# staffCd3_df = pd.DataFrame(staffCd_list3)
# staffCd3_df.to_csv('Output_Data/staffCd_2015_list3.csv', index=False)

,peopleCd,movieNm,peopleNm,repRoleNm,filmoNames
0,10004970,타짜: 원 아이드 잭,김영민,프로듀서,양자물리학|타짜: 원 아이드 잭|사라진 밤|그대 이름은 장미|두 남자|용의자|은밀하...
1,10002458,타짜: 원 아이드 잭,권오광,감독,타짜: 원 아이드 잭|범죄의 여왕|굿바이 싱글|돌연변이|소셜포비아|족구왕|세이프|질...
2,20331712,타짜: 원 아이드 잭,김재홍,촬영팀,타짜: 원 아이드 잭|벌새|증인|돌멩이
3,20146130,타짜: 원 아이드 잭,김보묵,아트디렉터,타짜: 원 아이드 잭|나랏말싸미|1987|불한당: 나쁜 놈들의 세상|싱글라이더|비밀...
4,20113179,타짜: 원 아이드 잭,김유경,프로듀서,타짜: 원 아이드 잭|사라진 밤|타짜-신의 손|아따쿨|블리치
...,...,...,...,...,...
109,20163358,극한직업,이지애,소품팀,기묘한 가족|극한직업|창궐|염력|대장 김창수|남한산성|보안관
110,10000853,극한직업,강정훈,소품팀,기묘한 가족|극한직업|창궐|7년의 밤|염력|대장 김창수|남한산성|섬. 사라진 사람들...
111,10004162,극한직업,김미혜,프로듀서,극한직업|리얼|파파|알포인트|피아노 치는 대통령
112,20227383,극한직업,김주연,예고편,나랏말싸미|기생충|어린 의뢰인|왓칭|사자|악질경찰|항거:유관순 이야기|극한직업|말모...


### # 주의 # 한 테스크가 끝날 때마다 csv로 저장하고 다음 작업 시작할 때는 저장된 csv 불러와서 시작해야 합니다.

In [19]:
directorCd_df = pd.read_csv('Output_Data/directorCd_2015_list.csv')
directorCd_df

,filmoNames,movieNm,peopleCd,peopleNm,repRoleNm
0,엑시트|사바하|군함도|군함도 감독판|베테랑|베를린|평양성|부당거래|해결사|다찌마와리...,베테랑,10019069,류승완,감독
1,"도청(가제)|암살|도둑들|전우치|중천|타짜|소년, 천국에 가다|그때 그 사람들|범죄...",암살,10072187,최동훈,감독
2,남산의 부장들|마약왕|내부자들: 디 오리지널|내부자들|간첩|파괴된 사나이|누가 예수...,내부자들,20111283,우민호,감독
3,"사바하|시간위의 집|검은 사제들|광해, 왕이 된 남자|특수본|바다|12번째 보조사제...",검은 사제들,10060462,장재현,감독
4,자산어보|변산|박열|대배우|동주|그날의 분위기|사도|소원|봄날의 입맞춤|평양성|구르...,사도,10057485,이준익,감독
5,연평해전|비디오를 보는 남자|민들레,연평해전,10006188,김학순,감독
6,드림(가제)|극한직업|레슬러|바람 바람 바람|스물|오늘의 연애|타짜-신의 손|힘내세...,스물,20125862,이병헌,감독
7,히말라야|해적: 바다로 간 산적|뜨거운 안녕|댄싱퀸|두 얼굴의 여친|방과후 옥상|이...,히말라야,10055965,이석훈,감독
8,조선명탐정: 흡혈괴마의 비밀|조선명탐정 : 사라진 놉의 딸|조선명탐정 : 각시투구꽃...,조선명탐정 : 사라진 놉의 딸,10004484,김석윤,감독
9,장사리 : 잊혀진 영웅들|암수살인|희생부활자|극비수사|친구 2|미운오리새끼|통증|굿...,극비수사,10001931,곽경택,감독


In [40]:
actorCd_df = pd.read_csv('Output_Data/actorCd_2015_list.csv')

In [32]:
writerCd_df = pd.read_csv('Output_Data/writerCd_2015_list.csv')

print(writerCd_df.shape)
writerCd_df

(194, 5)


,filmoNames,movieNm,peopleCd,peopleNm,repRoleNm
0,엑시트|사바하|군함도|군함도 감독판|베테랑|베를린|평양성|부당거래|해결사|다찌마와리...,베테랑,10019069,류승완,감독
1,도청(가제)|암살|도둑들|가위|난시청지역,암살,20128927,이기철,시나리오(각본)
2,"도청(가제)|암살|도둑들|전우치|중천|타짜|소년, 천국에 가다|그때 그 사람들|범죄...",암살,10072187,최동훈,감독
3,남산의 부장들|마약왕|내부자들: 디 오리지널|내부자들|간첩|파괴된 사나이|누가 예수...,내부자들,20111283,우민호,감독
4,"사바하|시간위의 집|검은 사제들|광해, 왕이 된 남자|특수본|바다|12번째 보조사제...",검은 사제들,10060462,장재현,감독
5,나랏말싸미|그날의 분위기|사도|평양성|페스티발|부당거래|구르믈 버서난 달처럼|불신지...,사도,10067370,조철현,제작
6,나랏말싸미|소녀의 세계|썬키스 패밀리|꾼|사도|미스체인지|평양성|부당거래|구르믈 버...,사도,20111297,오승현,제작
7,나랏말싸미|사도,사도,20188854,이송원,시나리오(각본)
8,드림(가제)|극한직업|레슬러|바람 바람 바람|스물|오늘의 연애|타짜-신의 손|힘내세...,스물,20125862,이병헌,감독
9,히말라야|스파이,히말라야,20185670,수오,시나리오(각본)


In [ ]:
# staffCd1_df = pd.read_csv('Output_Data/staffCd_2015_list1.csv')

In [ ]:
# staffCd2_df = pd.read_csv('Output_Data/staffCd_2015_list2.csv')

In [171]:
# staffCd3_df = pd.read_csv('Output_Data/staffCd_2015_list3.csv')

<br>\
<br>\
<br>
### 영화인 상세 가져오기

대망의 영화인 상세 데이터입니다.

In [33]:
def get_staff_details(peopleCd):
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json'

    # 영화인 코드를 이용해 세부 데이터를 불러옵니다.
    payload = {
        'key': key[5]
        , 'peopleCd': peopleCd
    }

    res = requests.get(url, params=payload).json()
    
    return res['peopleInfoResult']['peopleInfo']

TODO) 아래 테스트는 임시 데이터로 한 테스트 입니다. 위에서 Q4번이 해결 됐으면 해당 csv로 치환해주세요.

In [42]:
# 데이터 확인
directorCd_df

,filmoNames,movieNm,peopleCd,peopleNm,repRoleNm
0,엑시트|사바하|군함도|군함도 감독판|베테랑|베를린|평양성|부당거래|해결사|다찌마와리...,베테랑,10019069,류승완,감독
1,"도청(가제)|암살|도둑들|전우치|중천|타짜|소년, 천국에 가다|그때 그 사람들|범죄...",암살,10072187,최동훈,감독
2,남산의 부장들|마약왕|내부자들: 디 오리지널|내부자들|간첩|파괴된 사나이|누가 예수...,내부자들,20111283,우민호,감독
3,"사바하|시간위의 집|검은 사제들|광해, 왕이 된 남자|특수본|바다|12번째 보조사제...",검은 사제들,10060462,장재현,감독
4,자산어보|변산|박열|대배우|동주|그날의 분위기|사도|소원|봄날의 입맞춤|평양성|구르...,사도,10057485,이준익,감독
5,연평해전|비디오를 보는 남자|민들레,연평해전,10006188,김학순,감독
6,드림(가제)|극한직업|레슬러|바람 바람 바람|스물|오늘의 연애|타짜-신의 손|힘내세...,스물,20125862,이병헌,감독
7,히말라야|해적: 바다로 간 산적|뜨거운 안녕|댄싱퀸|두 얼굴의 여친|방과후 옥상|이...,히말라야,10055965,이석훈,감독
8,조선명탐정: 흡혈괴마의 비밀|조선명탐정 : 사라진 놉의 딸|조선명탐정 : 각시투구꽃...,조선명탐정 : 사라진 놉의 딸,10004484,김석윤,감독
9,장사리 : 잊혀진 영웅들|암수살인|희생부활자|극비수사|친구 2|미운오리새끼|통증|굿...,극비수사,10001931,곽경택,감독


In [43]:
# 임시로 테스트 데이터를 하나 불러와 저장했습니다.
# 어떤 데이터들이 어떤 형식으로 뽑혀나오는지 확인하고 상세정보를 어떻게 저장할지 설계합니다.
# 성별만 가져올까 하다가 여배우 남배우의 필모리스트도 비교해보면 좋을 듯 하여 저장하겠습니다.
get_staff_detail_test = get_staff_details(20167301)
get_staff_detail_test

{'peopleCd': '20167301',
 'peopleNm': '김대현',
 'peopleNmEn': 'KIM Dae-hyun',
 'sex': '남자',
 'repRoleNm': '감독',
 'homepages': [],
 'filmos': [{'movieCd': '20167821', 'movieNm': '시간의 종말', 'moviePartNm': '감독'},
  {'movieCd': '20167821', 'movieNm': '시간의 종말', 'moviePartNm': '시나리오(각본)'},
  {'movieCd': '20167821', 'movieNm': '시간의 종말', 'moviePartNm': '프로듀서'},
  {'movieCd': '20153303', 'movieNm': '다방의 푸른 꿈', 'moviePartNm': '감독'},
  {'movieCd': '20153303', 'movieNm': '다방의 푸른 꿈', 'moviePartNm': '시나리오(각본)'},
  {'movieCd': '20153303', 'movieNm': '다방의 푸른 꿈', 'moviePartNm': '프로듀서'},
  {'movieCd': '20153303', 'movieNm': '다방의 푸른 꿈', 'moviePartNm': '기획'},
  {'movieCd': '20128611', 'movieNm': '한국번안가요사', 'moviePartNm': '감독'},
  {'movieCd': '20100981', 'movieNm': '살인의 강', 'moviePartNm': '감독'},
  {'movieCd': '20100981', 'movieNm': '살인의 강', 'moviePartNm': '배우'},
  {'movieCd': '20100981', 'movieNm': '살인의 강', 'moviePartNm': '시나리오(각본)'},
  {'movieCd': '20180215', 'movieNm': '내 신발에게', 'moviePartNm': '감독'}]}

In [44]:
# 영화인 코드
get_staff_detail_test['peopleCd']

'20167301'

In [45]:
# 성별
get_staff_detail_test['sex']

'남자'

In [22]:
# 필모그래피 리스트
for movie in get_staff_detail_test['filmos']:
    print(movie['movieCd'])
    print(movie['movieNm'])
    print(movie['moviePartNm'])

NameError: name 'get_staff_detail_test' is not defined

In [23]:
# 2019년만 해도 감독, 배우, 스태프의 총 합은 3000을 훨씬 웃도는 숫자 입니다.
# 횟수 제한 문제로 3000번을 기준으로 잘라서 정보들을 저장한 후 나중에 합쳐주도록 하겠습니다.
# key 사용횟수가 3000회 이하로 남아있다면 가능 횟수에 맞춰서 코드 조정해주세요.
# 중간에 key 사용 가능 횟수가 모두 끝나면 저장하고 다른날 다시 불러와서 이어서 작업 해주세요.

director_detail_list = []
num = 0

for staffCd in directorCd_df['peopleCd'].iloc[:3000]:
    get_staff_detail_test = get_staff_details(staffCd)
    
    # 성별 리스트와 필모리스트를 따로 만들까 하다가 한 사람의 세부 정보인데 굳이 그럴 필요가 있나 싶어서 한꺼번에 저장합니다.
    # 필모리스트에 있는 영화들을 기준으로 해당 영화인이 참여 했던 작품 하나당 데이터 하나가 생성됩니다.
    # 데이터 한 행에는 '영화인코드, 이름, 성별, 참여영화코드, 참여영화이름, 참여했던역할(감독, 배우, 스태프 등)'이 들어갑니다.
    for movie in get_staff_detail_test['filmos']:
        person_info = {}

        person_info['peopleCd'] = get_staff_detail_test['peopleCd']
        person_info['peopleNm'] = get_staff_detail_test['peopleNm']
        person_info['sex'] = get_staff_detail_test['sex']

        person_info['movieCd'] = movie['movieCd']
        person_info['movieNm'] = movie['movieNm']
        person_info['moviePartNm'] = movie['moviePartNm']

        director_detail_list.append(person_info)
        
    print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])
    print(num)
    num += 1

NameError: name 'directorCd_df' is not defined

In [52]:
# 구간 바꿔서 다시 반복.
# get_staff_details 함수가 정의된 곳으로 돌아가 아직 사용 가능한 key 값으로 변경해줍니다.

actor_detail_list = []
num = 0

for staffCd in actorCd_df['peopleCd'].iloc[:3000]:
    get_staff_detail_test = get_staff_details(staffCd)
    
    for movie in get_staff_detail_test['filmos']:
        person_info = {}
        
        print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])

        person_info['peopleCd'] = get_staff_detail_test['peopleCd']
        person_info['peopleNm'] = get_staff_detail_test['peopleNm']
        person_info['sex'] = get_staff_detail_test['sex']

        person_info['movieCd'] = movie['movieCd']
        person_info['movieNm'] = movie['movieNm']
        person_info['moviePartNm'] = movie['moviePartNm']
        
        actor_detail_list.append(person_info)
                
    print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])
    print(num)
    num += 1

황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
0
유아인 남자
유아인 남자
유아인 남자
유아인 남자
유아인 남자
유아인 남자
유아인 남자
유아인 남자
유아인 남자
유아인 남자
유아인 남자
유아인 남자
유아인 남자
유아인 남자
유아인 남자
유아인 남자
1
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
2
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자
오달수 남자


고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
고규필 남자
40
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
서현우 남자
41
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
강영묵 남자
42
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 남자
한수현 

김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
85
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
배성우 남자
86
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
조재윤 남자
87
김대명 남자
김대명 남자
김대명 남자
김대명 남자
김대명 남자
김대명 남자
김대명 남자
김대명 남자
김대명 남자
김대명 남자
김대명 남자
김대명 남자
김대명 남자
김대명 남자
김대명 남자
김대명 남자
김대명 남자
88
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
조우진 남자
89
이엘 여자
이엘 여자
이엘 여자
이엘 여자
이엘 여자
이엘 여자
이엘 여자
이엘 여자
이엘 여자
이엘 여자
이엘 여자
이엘 여자
90
정만식 남자
정만식 남자
정만식 남자


김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
김해숙 여자
142
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
박원상 남자
143
진지희 여자
진지희 여자
진지희 여자
진지희 여자
진지희 여자
진지희 여자
진지희 여자
진지희 여자
진지희 여자
진지희 여자
144
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
박소담 여자
145
서예지 여자
서예지 여자
서예지 여자
서예지 여자
서예지 여자
서예지 여자
서예지 여자
서예지 여자
서예지 여자
146
이효제 남자
이효제 남자
이효제 남자
이효제 남자
이효제 남자
이효제 남자
이효제 남자
이효제 남자
147
곽동호 남자
곽동호 남자
148
최희영 여자
최희영 여자
149
정호 남자
정호 남자
150
이현철 남자
이현철 남자
151
김상훈 남자
김상훈 남자
김상훈 남자
김상훈 남자
152
이민혜 여자
이민혜 여자
153
김경

차민지 여자
차민지 여자
차민지 여자
차민지 여자
차민지 여자
차민지 여자
차민지 여자
210
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
황정민 남자
211
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
서정우 남자
212
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
조성하 남자
213
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 남자
김인권 

...

조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
조영진 남자
1892
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
양지웅 남자
1893
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
김태준 남자
1894
백도빈 남자
백도빈 남자
백도빈 남자
백도빈 남자
백도빈 남자
백도빈 남자
백도빈 남자
백도빈 남자
백도빈 남자
백도빈 남자
1895
송은채 여자
송은채 여자
송은채 여자
송은채 여자
송은채 여자
송은채 여자
송은채 여자
송은채 여자
1896
여욱환 남자
여욱환 남자
여욱환 남자
여욱환 남자
1897
남경읍 남자
남경읍 남자
남경읍 남자
남경읍 남자
남경읍 남자
남경읍 남자
남경읍 남자
남경읍 남자
남경읍 남자
남경읍 남자
1898
이종민 남자
이종민 남자
이종민 남자
이종민 남자
이종민 남자
1899
이탁건 남자
이탁건 남자
1900
임준혁 남자
임준혁 남자
1901
장현희 여자
장현희 여자
1902
정서경 여자


오재은 여자
오재은 여자
1969
김동현 남자
김동현 남자
김동현 남자
김동현 남자
김동현 남자
김동현 남자
김동현 남자
김동현 남자
김동현 남자
김동현 남자
김동현 남자
1970
김동현 남자
김동현 남자
김동현 남자
김동현 남자
김동현 남자
김동현 남자
김동현 남자
김동현 남자
김동현 남자
김동현 남자
김동현 남자
1971
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
김소연 여자
1972
김수정 여자
김수정 여자
김수정 여자
김수정 여자
김수정 여자
김수정 여자
김수정 여자
김수정 여자
1973
김여정 여자
김여정 여자
김여정 여자
김여정 여자
김여정 여자
김여정 여자
김여정 여자
김여정 여자
1974
김예은 여자
김예은 여자
김예은 여자
김예은 여자
김예은 여자
김예은 여자
김예은 여자
김예은 여자
김예은 여자
김예은 여자
김예은 여자
김예은 여자
김예은 여자
김예은 여자
김예은 여자
김예은 여자
김예은 여자
1975
김진주 여자
김진주 여자
김진주 여자
김진주 여자
김진주 여자
김진주 여자
김진주 여자
김진주 여자
김진주 여자
김진주 여자
1976
안동훈 남자
안동훈 남자
1977
신지은 여자
신지은 여자
1978
김희령 여자
김희령 여자
김희령 여자
김희령 여자
1979
송효정 여자
송효정 여자
송효정 여자
1980
송승민 남자
송승민 남자
1981
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 남자
류시문 

성호준 남자
성호준 남자
성호준 남자
성호준 남자
성호준 남자
성호준 남자
성호준 남자
2082
홍혜미 여자
홍혜미 여자
2083
서은지 여자
서은지 여자
서은지 여자
서은지 여자
2084
이지향 여자
이지향 여자
이지향 여자
이지향 여자
이지향 여자
2085
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
윤다경 여자
2086
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
길해연 여자
2087
안지혜 여자
안지혜 여자
안지혜 여자
2088
김선빈 남자
김선빈 남자
김선빈 남자
김선빈 남자
김선빈 남자
김선빈 남자
김선빈 남자
김선빈 남자
2089
곽민호 남자
곽민호 남자
곽민호 남자
곽민호 남자
곽민호 남자
곽민호 남자
곽민호 남자
곽민호 남자
곽민호 남자
곽민호 남자
곽민호 남자
곽민호 남자
곽민호 남자
곽민호 남자
곽민호 남자
2090
정준교 남자
정준교 남자
2091
이하민 여자
이하민 여자
2092
박형준 남자
박형준 남자
박형준 남자
박형준 남자
박형준 남자
2093
정완희 남자
정완희 남자
정완희 남자
2094
강필선 남자
강필선 남자
강필선 남자
강필선 남자
강필선 남자
강필선 남자
강필선 남자
강필선 남자
강필선 남자
강필선 남자
강필선 남자
강필선 남자
2095
공민정 여자
공민정 여자
공민정 여자
공민정 여자
공민정 여자
공민정 여자

백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
백재호 남자
2192
안지혜 여자
안지혜 여자
안지혜 여자
안지혜 여자
안지혜 여자
안지혜 여자
안지혜 여자
안지혜 여자
안지혜 여자
안지혜 여자
안지혜 여자
안지혜 여자
안지혜 여자
2193
현지윤 여자
현지윤 여자
2194
이새별 여자
이새별 여자
이새별 여자
이새별 여자
이새별 여자
이새별 여자
이새별 여자
이새별 여자
이새별 여자
이새별 여자
이새별 여자
이새별 여자
이새별 여자
이새별 여자
이새별 여자
이새별 여자
이새별 여자
이새별 여자
2195
김상석 남자
김상석 남자
김상석 남자
김상석 남자
김상석 남자
김상석 남자
김상석 남자
김상석 남자
김상석 남자
김상석 남자
김상석 남자
김상석 남자
김상석 남자
김상석 남자
2196
이종혁 남자
이종혁 남자
이종혁 남자
이종혁 남자
이종혁 남자
이종혁 남자
이종혁 남자
이종혁 남자
이종혁 남자
이종혁 남자
이종혁 남자
이종혁 남자
이종혁 남자
이종혁 남자
이종혁 남자
이종혁 남자
이종혁 남자
이종혁 남자
이종혁 남자
2197
강별 여자
강별 여자
강별 여자
강별 여자
강별 여자
2198
하연주 여자
하연주 여자
하연주 여자
2199
김형범 남자
김형범 남자
김형범 남자
김형범 남자
김형범 남자
김형범 남자
김형범 남자
김형범 남자
김형범 남자
김형범 남자
김형범 남자
김형범 남자
김형범 남자
2200
류현 남자
류현 남자
류현 남자
류현 남자
류현 남자
2201
이광모 남자
이광모 남자
2202
조선묵 남자
조선묵 남자
조선묵 남자
조선묵 남자
조선묵 남자
조선묵 남자
조선묵 남자
조선묵 남자
조선묵 남자
조선묵 남자
조선묵 남자
조선묵 남자
조선묵 남자
조선묵 남자
조선묵 남자
조선묵 남자
조선묵

임정은 여자
임정은 여자
2283
박선형 여자
박선형 여자
2284
변지현 여자
변지현 여자
2285
서정원 여자
서정원 여자
2286
서지형 남자
서지형 남자
2287
윤상호 남자
윤상호 남자
2288
윤정훈 남자
윤정훈 남자
2289
임정숙 여자
임정숙 여자
2290
장재권 남자
장재권 남자
장재권 남자
2291
전유성 남자
전유성 남자
2292
조지혜 여자
조지혜 여자
2293
최수혁 남자
최수혁 남자
2294
최승하 여자
최승하 여자
2295
한민호 남자
한민호 남자
한민호 남자
2296
홍선기 남자
홍선기 남자
2297
윤금선아 여자
윤금선아 여자
윤금선아 여자
윤금선아 여자
윤금선아 여자
윤금선아 여자
윤금선아 여자
윤금선아 여자
윤금선아 여자
윤금선아 여자
윤금선아 여자
윤금선아 여자
윤금선아 여자
윤금선아 여자
윤금선아 여자
윤금선아 여자
윤금선아 여자
윤금선아 여자
2298
김가나 여자
김가나 여자
2299
김성화 남자
김성화 남자
2300
김시유 남자
김시유 남자
2301
김혜령 여자
김혜령 여자
2302
양태열 남자
양태열 남자
2303
양혜정 여자
양혜정 여자
2304
이승언 남자
이승언 남자
2305
유하준 남자
유하준 남자
유하준 남자
유하준 남자
유하준 남자
유하준 남자
유하준 남자
2306
한제인 여자
한제인 여자
한제인 여자
한제인 여자
한제인 여자
한제인 여자
한제인 여자
한제인 여자
한제인 여자
한제인 여자
한제인 여자
2307
강용규 남자
강용규 남자
강용규 남자
2308
김도연 남자
김도연 남자
김도연 남자
김도연 남자
2309
박영훈 남자
박영훈 남자
박영훈 남자
박영훈 남자
박영훈 남자
박영훈 남자
박영훈 남자
2310
김정곤 남자
김정곤 남자
김정곤 남자
김정곤 남자
김정곤 남자
김정곤 남자
김정곤 남자
김정곤 남자
김정곤 남자
김정곤 남자
김정곤 남자
2311
문주연 여자
문주연 여자
문주연 여자
문주연 여자
문주연 여자
2312
최준열 남자
최준열 남자
2313
김민기 남자
김민기 남자
김민기 남자
김민기 남자
김

In [34]:
# 시나리오
# get_staff_details 함수가 정의된 곳으로 돌아가 아직 사용 가능한 key 값으로 변경해줍니다.

writer_detail_list = []
num = 0

for staffCd in writerCd_df['peopleCd']:
    get_staff_detail_test = get_staff_details(staffCd)
    
    for movie in get_staff_detail_test['filmos']:
        person_info = {}
        
        print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])

        person_info['peopleCd'] = get_staff_detail_test['peopleCd']
        person_info['peopleNm'] = get_staff_detail_test['peopleNm']
        person_info['sex'] = get_staff_detail_test['sex']

        person_info['movieCd'] = movie['movieCd']
        person_info['movieNm'] = movie['movieNm']
        person_info['moviePartNm'] = movie['moviePartNm']
        
        writer_detail_list.append(person_info)
                
    print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])
    print(num)
    num += 1

류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
류승완 남자
0
이기철 남자
이기철 남자
이기철 남자
이기철 남자
이기철 남자
이기철 남자
1
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
최동훈 남자
2
우민호 남자
우민호 남자
우민호 남자
우민호 남자
우민호 남자
우민호 남자
우민호 남자
우민호 남자
우민호 남자
우민호 남자
우민호 남자
우민호 남자
우민호 남자
우민호 남자
우민호 남자
3
장재현 남자
장재현 남자
장재현 남자
장재현 남자
장재현 남자
장재현 남자
장재현 남자
장재현 남자
장재현 남자
장재현 남자
장재현 남자
장재현 남자
장재현 남자
장재현 남자
장재현 남자
장재현 남자
장재현 남자
장재현 남자
장재현 남자
장재현 남자
4
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
조철현 남자
5
오승현 남자
오승현 남자
오승현 남자
오

조정화 None
조정화 None
조정화 None
조정화 None
조정화 None
조정화 None
조정화 None
조정화 None
92
신현덕 남자
신현덕 남자
신현덕 남자
신현덕 남자
신현덕 남자
신현덕 남자
신현덕 남자
신현덕 남자
신현덕 남자
신현덕 남자
신현덕 남자
93
박상글 여자
박상글 여자
박상글 여자
박상글 여자
박상글 여자
박상글 여자
94
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
김수훈 남자
95
최가희 여자
최가희 여자
96
최인희 여자
최인희 여자
최인희 여자
최인희 여자
97
조치언 남자
조치언 남자
조치언 남자
조치언 남자
조치언 남자
조치언 남자
조치언 남자
조치언 남자
조치언 남자
조치언 남자
98
정상선 남자
정상선 남자
99
안국진 남자
안국진 남자
안국진 남자
안국진 남자
안국진 남자
안국진 남자
안국진 남자
안국진 남자
안국진 남자
안국진 남자
안국진 남자
안국진 남자
안국진 남자
100
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
장건재 남자
101
이언희 여자
이언희 여자
이언희 여자
이언희 여자
이언희 여자
이언희 여자
이언희 여자
이언희 여자
이언희 여자
이언희 여자
이언희 여자
이언희 여자
102
이권 남자
이권 남자
이권 남자
이권 남자
이권 남자
이권 남자
이권 남자
이권 남자
이권 남자
이권 남자
이권 남자
이권 남자
이권 남자
103
변혜주 여자
변혜주 여자
변혜주 여자
변혜주 여자
변혜주 여

박준범 남자
박준범 남자
박준범 남자
박준범 남자
박준범 남자
박준범 남자
박준범 남자
박준범 남자
박준범 남자
박준범 남자
172
임완기 남자
임완기 남자
임완기 남자
173
이귀운 남자
이귀운 남자
174
류성 여자
류성 여자
175
김철민 남자
김철민 남자
김철민 남자
김철민 남자
김철민 남자
김철민 남자
김철민 남자
김철민 남자
176
여필주 None
여필주 None
177
정단우 남자
정단우 남자
정단우 남자
정단우 남자
정단우 남자
정단우 남자
178
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
김선 남자
179
이윤택 남자
이윤택 남자
180
이난 남자
이난 남자
이난 남자
이난 남자
이난 남자
이난 남자
이난 남자
이난 남자
이난 남자
이난 남자
이난 남자
이난 남자
이난 남자
이난 남자
이난 남자
이난 남자
이난 남자
181
황상원 남자
황상원 남자
황상원 남자
황상원 남자
182
김인규 남자
김인규 남자
김인규 남자
김인규 남자
김인규 남자
김인규 남자
183
강의석 남자
강의석 남자
강의석 남자
강의석 남자
강의석 남자
강의석 남자
강의석 남자
강의석 남자
강의석 남자
강의석 남자
184
권초원 None
권초원 None
185
이창열 남자
이창열 남자
이창열 남자
이창열 남자
186
동동이 남자
동동이 남자
187
조조 히데오 남자
조조 히데오 남자
조조 히데오 남자
조조 히데오 남자
조조 히데오 남자
조조 히데오 남자
조조 히데오 남자
조조 히데오 남자
조조 히데오 남자
조조 히데오 남자
조조 히데오 남자
조조 히데오 

In [35]:
# 가져온 데이터 확인

writer_detail_list

[{'peopleCd': '10019069',
  'peopleNm': '류승완',
  'sex': '남자',
  'movieCd': '20184621',
  'movieNm': '엑시트',
  'moviePartNm': '제작'},
 {'peopleCd': '10019069',
  'peopleNm': '류승완',
  'sex': '남자',
  'movieCd': '20177539',
  'movieNm': '사바하',
  'moviePartNm': '제작'},
 {'peopleCd': '10019069',
  'peopleNm': '류승완',
  'sex': '남자',
  'movieCd': '20161481',
  'movieNm': '군함도',
  'moviePartNm': '감독'},
 {'peopleCd': '10019069',
  'peopleNm': '류승완',
  'sex': '남자',
  'movieCd': '20161481',
  'movieNm': '군함도',
  'moviePartNm': '시나리오(각본)'},
 {'peopleCd': '10019069',
  'peopleNm': '류승완',
  'sex': '남자',
  'movieCd': '20161481',
  'movieNm': '군함도',
  'moviePartNm': '기획'},
 {'peopleCd': '10019069',
  'peopleNm': '류승완',
  'sex': '남자',
  'movieCd': '20177063',
  'movieNm': '군함도 감독판',
  'moviePartNm': '감독'},
 {'peopleCd': '10019069',
  'peopleNm': '류승완',
  'sex': '남자',
  'movieCd': '20148048',
  'movieNm': '베테랑',
  'moviePartNm': '감독'},
 {'peopleCd': '10019069',
  'peopleNm': '류승완',
  'sex': '남자',
  'movieCd'

In [ ]:
# # 구간 바꿔서 다시 반복.
# # get_staff_details 함수가 정의된 곳으로 돌아가 아직 사용 가능한 key 값으로 변경해줍니다.

# staff_detail_list_1 = []
# num = 1

# for staffCd in staffCd1_df['peopleCd'].iloc[:3000]:
#     get_staff_detail_test = get_staff_details(staffCd)
    
#     for movie in get_staff_detail_test['filmos']:
#         person_info = {}
        
#         print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])

#         person_info['peopleCd'] = get_staff_detail_test['peopleCd']
#         person_info['peopleNm'] = get_staff_detail_test['peopleNm']
#         person_info['sex'] = get_staff_detail_test['sex']

#         person_info['movieCd'] = movie['movieCd']
#         person_info['movieNm'] = movie['movieNm']
#         person_info['moviePartNm'] = movie['moviePartNm']
        
#         staff_detail_list_1.append(person_info)
                
#     print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])
#     print(num)
#     num += 1

In [ ]:
# # 구간 바꿔서 다시 반복.
# # get_staff_details 함수가 정의된 곳으로 돌아가 아직 사용 가능한 key 값으로 변경해줍니다.

# staff_detail_list_2 = []
# num = 1

# for staffCd in staffCd1_df['peopleCd'].iloc[:3000]:
#     get_staff_detail_test = get_staff_details(staffCd)
    
#     for movie in get_staff_detail_test['filmos']:
#         person_info = {}
        
#         print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])

#         person_info['peopleCd'] = get_staff_detail_test['peopleCd']
#         person_info['peopleNm'] = get_staff_detail_test['peopleNm']
#         person_info['sex'] = get_staff_detail_test['sex']

#         person_info['movieCd'] = movie['movieCd']
#         person_info['movieNm'] = movie['movieNm']
#         person_info['moviePartNm'] = movie['moviePartNm']
        
#         staff_detail_list_2.append(person_info)
                
#     print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])
#     print(num)
#     num += 1

In [ ]:
# # 구간 바꿔서 다시 반복.
# # get_staff_details 함수가 정의된 곳으로 돌아가 아직 사용 가능한 key 값으로 변경해줍니다.

# staff_detail_list_3 = []
# num = 1

# for staffCd in staffCd1_df['peopleCd'].iloc[:3000]:
#     get_staff_detail_test = get_staff_details(staffCd)
    
#     for movie in get_staff_detail_test['filmos']:
#         person_info = {}
        
#         print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])

#         person_info['peopleCd'] = get_staff_detail_test['peopleCd']
#         person_info['peopleNm'] = get_staff_detail_test['peopleNm']
#         person_info['sex'] = get_staff_detail_test['sex']

#         person_info['movieCd'] = movie['movieCd']
#         person_info['movieNm'] = movie['movieNm']
#         person_info['moviePartNm'] = movie['moviePartNm']
        
#         staff_detail_list_3.append(person_info)
                
#     print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])
#     print(num)
#     num += 1

<br>

### TODO. 영화인 상세정보를 뽑아주세요.

Q1) 2015 감독, 배우, 스태프 모두의 상세정보를 뽑아주세요.
각각의 리스트를 합칠 때는 아래 코드를 참고하세요.

```python
staff_detail_list_all = pd.DataFrame(staff_detail_list_1 + staff_detail_list_2 + staff_detail_list_3 + staff_detail_list_4 + staff_detail_list_5)
staff_detail_list_all
```

Q2) 가져온 데이터는 csv 형식, index=False로 저장해주세요.

In [49]:
director_detail_df = pd.DataFrame(director_detail_list)
director_detail_df.to_csv('Output_Data/director_detail_2015_ko.csv', index=False)

In [53]:
actor_detail_df = pd.DataFrame(actor_detail_list)
actor_detail_df.to_csv('Output_Data/actor_detail_2015_ko.csv', index=False)

In [36]:
writer_detail_df = pd.DataFrame(writer_detail_list)
writer_detail_df.to_csv('Output_Data/writer_detail_2015_ko.csv', index=False)

In [ ]:
# staff1_detail_df = pd.DataFrame(staff_detail_list_1)
# staff1_detail_df.to_csv('Output_Data/staff1_detail_2015_ko.csv', index=False)

In [ ]:
# staff2_detail_df = pd.DataFrame(staff_detail_list_2)
# staff2_detail_df.to_csv('Output_Data/staff2_detail_2015_ko.csv', index=False)

In [ ]:
# staff3_detail_df = pd.DataFrame(staff_detail_list_3)
# staff3_detail_df.to_csv('Output_Data/staff3_detail_2015_ko.csv', index=False)

영화인 상세 잘 저장됐는지 확인

In [55]:
pd.read_csv('Output_Data/director_detail_2015_ko.csv')

,movieCd,movieNm,moviePartNm,peopleCd,peopleNm,sex
0,20184621,엑시트,제작,10019069,류승완,남자
1,20177539,사바하,제작,10019069,류승완,남자
2,20161481,군함도,감독,10019069,류승완,남자
3,20161481,군함도,시나리오(각본),10019069,류승완,남자
4,20161481,군함도,기획,10019069,류승완,남자
5,20177063,군함도 감독판,감독,10019069,류승완,남자
6,20148048,베테랑,감독,10019069,류승완,남자
7,20148048,베테랑,시나리오(각본),10019069,류승완,남자
8,20124083,베를린,감독,10019069,류승완,남자
9,20124083,베를린,시나리오(각본),10019069,류승완,남자


In [56]:
pd.read_csv('Output_Data/actor_detail_2015_ko.csv')

,movieCd,movieNm,moviePartNm,peopleCd,peopleNm,sex
0,20197922,다만 악에서 구하소서,배우,10090290,황정민,남자
1,20192194,인질(가제),배우,10090290,황정민,남자
2,20179101,공작,배우,10090290,황정민,남자
3,20161481,군함도,배우,10090290,황정민,남자
4,20177063,군함도 감독판,배우,10090290,황정민,남자
5,20153443,아수라,배우,10090290,황정민,남자
6,20140194,곡성,배우,10090290,황정민,남자
7,20145507,검사외전,배우,10090290,황정민,남자
8,20136068,히말라야,배우,10090290,황정민,남자
9,20148048,베테랑,배우,10090290,황정민,남자


<br>

### 잘 돌아가다가 KeyError: 'peopleInfoResult'가 발생했을 때.

해당에러가 뜰 경우에는 key 사용 한도를 초과한 것입니다.

멈추기 전까지는 데이터가 다 저장되어있으니 당황하지 마시고 어디까지 저장되었는지 아래처럼 범위를 이용해서 또는 .tail() 함수를 이용해 데이터를 확인하고 다시 시작해야하는 부분이 어디인지 알아냅니다.

In [147]:
# 데이터 확인
staff_detail_list_2[-10:]

[{'peopleCd': '10006383',
  'peopleNm': '김혜연',
  'sex': '여자',
  'movieCd': '20040586',
  'movieNm': '돌려차기',
  'moviePartNm': '헤어팀'},
 {'peopleCd': '10006383',
  'peopleNm': '김혜연',
  'sex': '여자',
  'movieCd': '20040586',
  'movieNm': '돌려차기',
  'moviePartNm': '특수분장'},
 {'peopleCd': '10006383',
  'peopleNm': '김혜연',
  'sex': '여자',
  'movieCd': '20030107',
  'movieNm': '거울속으로',
  'moviePartNm': '분장팀'},
 {'peopleCd': '10006383',
  'peopleNm': '김혜연',
  'sex': '여자',
  'movieCd': '20030107',
  'movieNm': '거울속으로',
  'moviePartNm': '헤어팀'},
 {'peopleCd': '10006383',
  'peopleNm': '김혜연',
  'sex': '여자',
  'movieCd': '20020118',
  'movieNm': '재밌는 영화',
  'moviePartNm': '의상팀'},
 {'peopleCd': '10006383',
  'peopleNm': '김혜연',
  'sex': '여자',
  'movieCd': '20010031',
  'movieNm': '수취인불명',
  'moviePartNm': '의상팀'},
 {'peopleCd': '10006383',
  'peopleNm': '김혜연',
  'sex': '여자',
  'movieCd': '20158554',
  'movieNm': '고백',
  'moviePartNm': '분장'},
 {'peopleCd': '10006383',
  'peopleNm': '김혜연',
  'sex': '여자',
  'm

In [110]:
# '김효주', '아메리카 타운' 정보가 맨 마지막일 경우
# index값을 확인한다. 3000까지 뽑힌거니까 3001부터 데이터를 다시 가져오면 됨.
peopleCd_list[(peopleCd_list['peopleNm'] == '김효주') & (peopleCd_list['movieNm'] == '아메리카 타운')]

,peopleCd,movieNm,peopleNm,repRoleNm,filmoNames
3000,20320553,아메리카 타운,김효주,분장팀,아메리카 타운
